### Mutual Recursion

In [2]:
def sum(n: Int): Int =
    if (n <= 0) 0 else (n+sum(n-1))
sum(20000) // stack overflow

: 

In [4]:
import scala.annotation.tailrec

def sum(n: Int): Int = {
    @tailrec def sumItr(res: Int, m: Int): Int =
    if (m <= 0) res else sumItr(m+res,m-1)
    sumItr(0,n)
}
sum(20000)

import scala.annotation.tailrec


defined function sum
res3_2: Int = 200010000

In [1]:
def sum(acc: Int, n: Int): Int =
    if (n <= 0) acc else sum2(n + acc, n-1)
def sum2(acc: Int, n: Int): Int =
    if (n <= 0) acc else sum(2*n + acc, n-1)
sum(0, 20000) // stack overflow

: 

* ***.done***: `T -> TailRec[T]`
* ***.result***: `TailRec[T] -> T`

In [7]:
import scala.util.control.TailCalls._

def sum(acc: Int, n: Int): TailRec[Int] =
    if (n <= 0) done(acc) else tailcall(sum2(n + acc, n-1))
def sum2(acc: Int, n: Int): TailRec[Int] =
    if (n <= 0) done(acc) else tailcall(sum(2*n + acc, n-1))
sum(0, 20000).result

import scala.util.control.TailCalls._


defined function sum
defined function sum2
res6_3: Int = 300010000

### Currying
* from *Haskell Brooks Curry*

*Q. Is there any benefit using Currying*
> Convenience but not better performance

In [22]:
def sum(f: Int=>Int): (Int,Int)=>Int = {
    def sumF(a: Int, b: Int): Int =
        if (a <= b) f(a) + sumF(a+1, b) else 0
    sumF _
}
// sum(n=>n) w. closure
sum(n=>n)(0, 100)
def sumLinear = sum(n=>n)

defined function sum
res21_1: Int = 5050
defined function sumLinear

In [23]:
def sum(f: Int=>Int)(a: Int, b: Int): Int =
    if (a <= b) f(a) + sum(f)(a+1, b) else 0

// sum(n=>n) w.o. closure
sum(n=>n)(0, 100)
def sumLinear = sum(n=>n) _

defined function sum
res22_1: Int = 5050
defined function sumLinear

In [26]:
def a = 1
def b = a _

print(a)

1

defined function a
defined function b

#### foo example

In [31]:
def foo(x: Int, y: Int, z: Int)(a: Int, b: Int) =
    x + y + z + a + b

val f1 = (x: Int, z: Int, b: Int)=> foo(x,1,z)(2,b)
val f2 = foo(_:Int,1,_:Int)(2, _:Int)
val f3 = (x: Int, z: Int) => ((b: Int) => foo(x,1,z)(2,b))

f1(1,2,3)
f2(1,2,3)
f3(1,2)(3)

defined function foo
f1: (Int, Int, Int) => Int = ammonite.$sess.cmd30$Helper$$Lambda$2087/44385230@6356bef7
f2: (Int, Int, Int) => Int = ammonite.$sess.cmd30$Helper$$Lambda$2088/1732068708@336a24f1
f3: (Int, Int) => Int => Int = ammonite.$sess.cmd30$Helper$$Lambda$2089/597523141@2d1a60f1
res30_4: Int = 9
res30_5: Int = 9
res30_6: Int = 9

#### map reduce example

In [36]:
def mapReduce(reduce: (Int,Int)=>Int, inival: Int, f: Int=>Int, a: Int, b: Int): Int = {
    if (a <= b) reduce(f(a),mapReduce(reduce,inival,f,a+1,b))
    else inival
}

def sum(f: Int=>Int, a: Int, b: Int): Int =
    mapReduce((x,y)=>x+y,0,f,a,b)
def product(f: Int=>Int, a: Int, b: Int): Int =
    mapReduce((x,y)=>x*y,1,f,a,b)

sum(x=>x, 1, 100)

defined function mapReduce
defined function sum
defined function product
res35_3: Int = 5050

In [34]:
def mapReduce(reduce: (Int,Int)=>Int, inival: Int)(f: Int=>Int)(a: Int, b: Int): Int = {
    if (a <= b) reduce(f(a),mapReduce(reduce,inival)(f)(a+1,b))
    else inival
}

// need to make a closure since mapReduce is param. code.
def sum = mapReduce((x,y)=>x+y,0) _

// val is better than def. Think about why.
val product = mapReduce((x,y)=>x*y,1) _

sum(x=>x)(1, 100)

defined function mapReduce
defined function sum
product: Int => Int => (Int, Int) => Int = ammonite.$sess.cmd33$Helper$$Lambda$2124/2099624160@619055c3
res33_3: Int = 5050

### Exception

In [42]:
class factRangeException(val arg: Int) extends Exception

def fact(n : Int): Int =
    if (n < 0) throw new factRangeException(n)
    else if (n == 0) 1
    else n * fact(n-1)

def foo(n: Int) = fact(n + 10)

defined class factRangeException
defined function fact
defined function foo

In [43]:
// type Any 
try {
    100
} catch {
    case e : factRangeException => {
        println("fact range error: " + e.arg)
    }
}

6
fact range error: -90


res42: AnyVal = ()

In [48]:
val res: Int = 
    try {
        println(fact(3))
        println(foo(-100))
        100
    } catch {
        case e : factRangeException => {
            println("fact range error: " + e.arg)
        }
        200
    }

6
fact range error: -90


res: Int = 200

### Tuples
* types of elements can be different, but not in array
* static type (# of e cannot exceed 22)

In [51]:
def x = (1, "a", 10)
x._1
x._2
x._3

defined function x
res50_1: Int = 1
res50_2: String = "a"
res50_3: Int = 10

### Structural Types (a.k.a. Record Types)

In [54]:
                                    // E1[]
def bar (x: Int) = x+1              // E1[bar=(x)x+1]
val foo = new {                     // E1:E2[]
    val a = 1+2                     // E1:E2[a=3]
    def b = a + 1                   // E1:E2[a=3,b=a+1]
    def f(x: Int) = b + x           // E1:E2[a=3,b=a+1,f=(x)b+x]
    def f(x: String) = "hello" + x  // E1:E2[...]
    val g : Int => Int = bar _      // E1:E2[...]
}                                   // E1[bar=(x)x+1,foo=(E2)]
foo.b
foo.f(3)
foo.f("gil")

defined function bar
foo: {val a: Int;def b: Int;def f(x: Int): Int;def f(x: String): String;val g: Int => Int} = ammonite.$sess.cmd53$Helper$$anon$1@4c7719
res53_2: Int = 4
res53_3: Int = 7
res53_4: String = "hellogil"

In [55]:
def bar (x: Int) = x+1

object foo {
    val a = 1+2
    def b = a + 1
    def f(x: Int) = b + x
    def f(x: String) = "hello" + x
    val g : Int => Int = bar _
}
foo.b
foo.f(3)
foo.f("gil")

defined function bar
defined object foo
res54_2: Int = 4
res54_3: Int = 7
res54_4: String = "hellogil"

In [70]:
{
    val a = 1+2
    def b = a + 1
    def f(x: Int) = b + x
    
    // not allowed to duplicate names
    def f(x: String) = "hello" + x
    val g : Int => Int = bar _
}

a: Int = 3
defined function b
defined function f
defined function f
g: Int => Int = ammonite.$sess.cmd69$Helper$$Lambda$2875/807692961@2ba60eaf

* Not allowed to use same name in the block expression.
* But okay to use in structural type.

In [67]:
def g(x: {
    val a: Int
    def b: Int
    def f(x:Int): Int
    def f(x:String): String
    val g: Int => Int}) =
    x.f(3)

// ambiguous reference
//     x.f _
g(foo)

defined function g
res66_1: Int = 7

In [69]:
type Foo = {
    val a: Int
    def b: Int
    def f(x:Int): Int
    def f(x:String): String
    val g: Int => Int
}

def g(x: Foo) = x.f(3)
g(foo)

defined type Foo
defined function g
res68_2: Int = 7